# MLP

In [3]:
from mxnet import nd
from mxnet.gluon import nn

class MLP(nn.Block):
    def __init__(self, **kwargs):
        super(MLP, self).__init__(**kwargs)
        self.hidden = nn.Dense(256, activation='relu')
        self.output = nn.Dense(10)
        
    def forward(self, x):
        return self.output(self.hidden(x))

In [4]:
x = nd.random.uniform(shape=(2, 20))
net = MLP()
net.initialize()
net(x)


[[ 0.02394444  0.05150646  0.01035161 -0.06435342 -0.05801985  0.06419201
   0.04472664 -0.01852541 -0.03237377  0.0738937 ]
 [ 0.05207362  0.04186264  0.04021509 -0.06558425 -0.02249499  0.03413139
   0.02135914 -0.06898529  0.02329671  0.00336681]]
<NDArray 2x10 @cpu(0)>

# Sequential

In [5]:
class MySequential(nn.Block):
    def __init__(self, **kwargs):
        super(MySequential, self).__init__(**kwargs)
        
    def add(self, block):
        self._children[block.name] = block
    
    def forward(self, x):
        for block in self._children.values():
            x = block(x)
        return x

net = MySequential()
net.add(nn.Dense(256, activation='relu'))
net.add(nn.Dense(10))
net.initialize()
net(x)


[[-0.03358278  0.00098314  0.03334405 -0.00663612  0.07881726 -0.01704565
  -0.01302505 -0.0544973   0.0414939   0.00170794]
 [ 0.01879605 -0.04185785  0.02918838 -0.00970372  0.05835273 -0.03129902
  -0.00644607 -0.02542867  0.0442826   0.01446365]]
<NDArray 2x10 @cpu(0)>

# Fancy MLP

In [7]:
class FancyMLP(nn.Block):
    def __init__(self, **kwargs):
        super(FancyMLP, self).__init__(**kwargs)
        self.rand_weight = self.params.get_constant('rand_weight', nd.random.uniform(shape=(20, 20)))
        self.dense = nn.Dense(20, activation='relu')
    
    def forward(self, x):
        x = self.dense(x)
        x = nd.relu(nd.dot(x, self.rand_weight.data()) + 1)
        x = self.dense(x)
        
        while x.norm().asscalar() > 1:
            x /= 2
        if x.norm().asscalar() < 0.8:
            x *= 10
        return x.sum()
    
net = FancyMLP()
net.initialize()
net(x)


[3.0281014]
<NDArray 1 @cpu(0)>